In [1]:
import scipy as sci
import scipy.optimize
import numpy as np
import fractions as frac

Consider first the Market (B,S1,S2) given by:
\begin{align}
S_1^1=\begin{bmatrix}36\\24\\20\\12\end{bmatrix},S_1^2=\begin{bmatrix}72\\120\\96\\48\end{bmatrix}, B_1=(1+r)
\end{align}
Where:
\begin{align}
S_0^1=11, S_0^2=44, r=1
\end{align}

With probabilities:
\begin{align}
Q=[1/6,1/6,1/2,1/6]
\end{align}


## Q1: 
### (i)Is the market arbitrage free?

In [2]:
s11=[36,24,20,12]
s12=[72,120,96,48]
s01=11
s02=44
r=1
#find u and d for each underlying
u1=np.max(s11)/s01
d1=np.min(s11)/s01
u2=np.max(s12)/s02
d2=np.min(s12)/s02
#Check if the 1+r lies between d and u
print(d1<(1+r)<u1)
print(d2<(1+r)<u2)


True
True


In [3]:
#alternatively observe the set of equivalent martingale measures
A=np.array([[18,12,10,6],[36,60,48,24],[1,1,1,1]])
b=np.array([11,44,1])
q1=[0,1]
q2=[0,1]
q3=[0,1]
q4=[0,1]

for i in range(0,4):
    qi=np.zeros(4)
    qi[i]=1
    print("Lower bound of q%s"%i,(sci.optimize.linprog(qi,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[q1,q2,q3,q4]).x[i]))
    print("Upper bound of q%s"%i,(sci.optimize.linprog(-qi,A_eq=-A,b_eq=-b,options={"presolve" : "False"},bounds=[q1,q2,q3,q4]).x[i]))

Lower bound of q0 0.1666666666682874
Upper bound of q0 0.16666666666837707
Lower bound of q1 4.096089252087346e-13
Upper bound of q1 0.5000000000064818
Lower bound of q2 2.5586991552138736e-12
Upper bound of q2 0.75000000192873
Lower bound of q3 0.0833333333339302
Upper bound of q3 0.3333333333111472


From this we can see the set of Equivalent Martingale measures can be expressed as
\begin{align}
Q=\begin{bmatrix}1/6\\\frac{1}{2}-\frac{2}{3}t\\t\\\frac{1-t}{3}\end{bmatrix}, t\in (0,\frac{3}{4})
\end{align}
We can see then that the set of equivalent martingale measures is not a null set, so there is no arbitrage
(I don't know that the bounds actually show that but I don't know how else to represent the work in python)

### (ii)

Consider now the straddle option on $S^1$ with strike $K=24$, which has payoff:
\begin{align}
X_1=(S_1^1-24)^++(24-S1_1^1)^+ =\begin{bmatrix}12\\0\\4\\12\end{bmatrix}
\end{align}
Find the Arbitrage Free Price of this derivative

In [4]:
c=[0,0,0]
A=[[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]]
b=[6,0,2,6]
kb=[0,None]
ha=[None,None]
hb=[None,None]
sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,ha,hb])
#Here we can see that the arbitrage free price of the derivative is 3

<ipython-input-4-2b69e20d1bac>:7: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,ha,hb])


     con: array([ 3.70903308e-12, -7.40296713e-13,  7.43849426e-13,  3.71258579e-12])
     fun: 0.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 3. ,  0.2, -0.2])

Show that $U(X)=D(X)$

In [5]:
#U(X)
c=[1,0,0] #minimise x in (x,h1,h2) for super-replicating portfolios
A=np.array([[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]]) #set of values of (x(w),s1(w),s2(w))
b=np.array([6,0,2,6])
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_ub=-A,b_ub=-b,options={"presolve" : "False"},bounds=[kb,h1,h2])
#here we use -A and -b as the function uses "-Ax<-b" ie "Ax>b" which is what we want 

     con: array([], dtype=float64)
     fun: 2.9999999994237925
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 4.38754011e-09,  1.38527856e-10, -1.28437616e-09, -4.13018419e-09])
  status: 0
 success: True
       x: array([ 3. ,  0.2, -0.2])

In [6]:
#D(X)
c=[-1,0,0] #maximise x in (x,h1,h2) for super-replicating portfolios
A=np.array([[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]]) #set of values of (x(w),s1(w),s2(w))
b=np.array([6,0,2,6])
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_ub=A,b_ub=b,options={"presolve" : "False"},bounds=[kb,h1,h2])

     con: array([], dtype=float64)
     fun: -2.999999999617449
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 2.06582040e-09, -1.57786673e-10,  1.19495525e-11,  3.51422003e-10])
  status: 0
 success: True
       x: array([ 3. ,  0.2, -0.2])

Here we see $U(X)=D(X)$ which shows that the arbitrage free price of $X$, $X_0=3$

### (iii)
Is the market complete?

No as we saw above the Equivalent Martingale Measure is not unique, more generally the vector space given by
\begin{align}
\begin{bmatrix}1\\1\\1\\1\end{bmatrix},\begin{bmatrix}7\\1\\-1\\-5\end{bmatrix},\begin{bmatrix}-8\\16\\4\\-20\end{bmatrix}
\end{align}
do not span $\mathbb{R}^4$, the set containing all possible derivative payoffs

### (iv)

Now consider the 2 Basket Call Options Y1,Y2 on $S^1$ and $S^2$ with strike $K_1=31$ and $K_2=55$ respectively. The payoff is given by: 
\begin{align}
\left(\frac{(S_1^1+S_1^2)}{2}-K\right)^+
\end{align}

Are these options replicable?

\begin{align}
x\begin{bmatrix}1\\1\\1\\1\end{bmatrix}+h^1\begin{bmatrix}7\\1\\-1\\-5\end{bmatrix}+h^2\begin{bmatrix}-8\\16\\4\\-20\end{bmatrix}=\begin{bmatrix}\frac{23}{2}\\\frac{41}{2}\\\frac{27}{2}\\0\end{bmatrix}
\end{align}
\begin{align}
x\begin{bmatrix}1\\1\\1\\1\end{bmatrix}+h^1\begin{bmatrix}7\\1\\-1\\-5\end{bmatrix}+h^2\begin{bmatrix}-8\\16\\4\\-20\end{bmatrix}=\begin{bmatrix}0\\\frac{17}{2}\\\frac{3}{2}\\0\end{bmatrix}
\end{align}

In [7]:
#solving the above equation for Y1
c=[0,0,0]
A=[[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]]
b=[23/2,41/2,27/2,0]
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,h1,h2])
#We can see this has no solution, so is not replicable

<ipython-input-7-87ca27964321>:8: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,h1,h2])


     con: array([11.5, 20.5, 13.5,  0. ])
     fun: 0.0
 message: 'There is a linear combination of rows of A_eq that results in zero, suggesting a redundant constraint. However the same linear combination of b_eq is nonzero, suggesting that the constraints conflict and the problem is infeasible.'
     nit: 0
   slack: array([], dtype=float64)
  status: 2
 success: False
       x: array([0., 0., 0.])

In [8]:
#solving the above equation for Y2
c=[0,0,0]
A=[[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]]
b=[0,17/2,3/2,0]
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,h1,h2])
#We can see this has no solution, so is not replicable

<ipython-input-8-201870378c30>:8: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,h1,h2])


     con: array([0. , 8.5, 1.5, 0. ])
     fun: 0.0
 message: 'There is a linear combination of rows of A_eq that results in zero, suggesting a redundant constraint. However the same linear combination of b_eq is nonzero, suggesting that the constraints conflict and the problem is infeasible.'
     nit: 0
   slack: array([], dtype=float64)
  status: 2
 success: False
       x: array([0., 0., 0.])

So neither option is replicable

### (v)
Compute the set of Arbitrage Free Prices for $Y_1$

In [9]:
#First find U(x)
c=[1,0,0]
A=np.array([[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]])
b=np.array([23/2,41/2,27/2,0])
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_ub=-A,b_ub=-b,options={"presolve" : "False"},bounds=[kb,h1,h2])
#We can see this has no solution, so is not replicable

     con: array([], dtype=float64)
     fun: 12.166666660613027
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([ 2.07015560e-08, -5.31396225e-08,  1.66666648e-01,  5.11977358e-08])
  status: 0
 success: True
       x: array([12.16666666,  0.46666667,  0.49166666])

In [10]:
#Find D(x)
c=[-1,0,0]
A=np.array([[1,7,-8],[1,1,16],[1,-1,4],[1,-5,-20]])
b=np.array([23/2,41/2,27/2,0])
kb=[0,None]
h1=[None,None]
h2=[None,None]
sci.optimize.linprog(c,A_ub=A,b_ub=b,options={"presolve" : "False"},bounds=[kb,h1,h2])
#We can see this has no solution, so is not replicable

     con: array([], dtype=float64)
     fun: -12.041666548432314
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([1.65180550e-07, 2.50000108e-01, 1.08781116e-07, 1.09421091e-07])
  status: 0
 success: True
       x: array([12.04166655,  0.47499999,  0.48333333])

So the set of arbitrage free prices is ($12\frac{1}{24}$,$12\frac{1}{6}$)

### (vi)
Fix a fair price for $Y_1$, then in the expanded market $(B,S^1,S^2,Y^1)$ replicate $Y^2$

The set of equations for $Y^2$ becomes:
\begin{align}
x\begin{bmatrix}1\\1\\1\\1\end{bmatrix}+h^1\begin{bmatrix}7\\1\\-1\\-5\end{bmatrix}+h^2\begin{bmatrix}-8\\16\\4\\-20\end{bmatrix}+h^3\begin{bmatrix}-\frac{3}{5}\\\frac{42}{5}\\\frac{7}{5}\\-\frac{121}{10}\end{bmatrix}
=\begin{bmatrix}0\\\frac{17}{2}\\\frac{3}{2}\\0\end{bmatrix}
\end{align}
Where the $Y^1_0 = \frac{121}{10}$

In [11]:
c=[0,0,0,0]
A=[[1,7,-8,-3/5],[1,1,16,42/5],[1,-1,4,7/5],[1,-5,-20,-121/10]]
b=[0,17/2,3/2,0]
kb=[0,None]
h1=[None,None]
h2=[None,None]
h3=[None,None]
sci.optimize.linprog(c,A_eq=A,b_eq=b,options={"presolve" : "False"},bounds=[kb,h1,h2,h3])

     con: array([-7.41783879e-09,  5.56378676e-08,  3.70906683e-09, -7.41753325e-09])
     fun: 0.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([  2.58333332, -11.94999991, -12.00833324,  24.99999981])

$Y_0^2=\frac{31}{12}$

Would also be interesting to see a question about a super/sub hedge for Y1 or Y2 when they are not replicable, would this simply be U(Y2), I've seen some definitions requiring that the replicating portfolio is self-financing